In [ ]:
#Import all the libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.metrics import classification_report
from keras import layers, models, optimizers
import sklearn.metrics as metrics
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
project = pd.read_excel (r'Dataset')

In [ ]:
project.head()

In [ ]:
!pip install math
!pip install datatest

In [ ]:
from datatest import validate
from math import isnan

In [ ]:
project[project['تبصرے'] == 'Neative'].index

Int64Index([], dtype='int64')

In [ ]:

def removing_features(features, data):
  data.drop(features, inplace = True, axis = 1)

In [ ]:
project.head()

تبصرے  جذبات
0  تم جیسی اورت کو تو مار جان چاہے ہر جگ ہو ہو پھ...      0
1  میری حمیشہ سی تم خوحیش ہی ہے میرے لوگ  جھوٹ کھ...      1
2  توگے مجھ سے گھر میں غوث کر مڑو گا تغ جیسی اورٹ...      0
3  جب ہم اپکی باتیں سنت ہے تو دل مجھے اپکی عزت یا...      1
4  سالی رندی کی الاواد گشتی اورنگ ننگی ہی کو نہیں...      0

In [ ]:
# text cleaning and pre-processing:
def delete_urdu_english_symbols(project):
    cleaned = []
    for sentence in project:
        text = re.sub(r"\d+", " ", sentence)
        text = re.sub(r'\#\.', '', text)
        text = re.sub(r'\n', '', text)
        text = re.sub(r',', '',text)
        text = re.sub(r'\-', ' ',text)
        text = re.sub(r'\.', '',text)
        text = re.sub(r'\\', ' ',text)
        text = re.sub(r'\\x\.+', '',text)
        text = re.sub(r'\d', '',text)
        text = re.sub(r'^_.', '',text)
        text = re.sub(r'_', ' ',text)
        text = re.sub(r'^ ', '',text)
        text = re.sub(r' $', '',text)
        text = re.sub(r'\?', '',text)
        text = re.sub(r'é', '',text)
        text = re.sub(r'§', '',text)
        text = re.sub(r'¦', '',text)
        text = re.sub(r'æ', '',text)
        text = re.sub(r'\d+', '',text)
        text = re.sub('(.*?)\d+(.*?)', '',text)

        # English punctuations
        text = re.sub(r"""[!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]+""", " ", text)
        # Urdu punctuations
        text = re.sub(r"[:؛؟’‘٭ء،۔]+", " ", text)
        # Arabic numbers
        text = re.sub(r"[٠‎١‎٢‎٣‎٤‎٥‎٦‎٧‎٨‎٩]+", " ", text)
        text = re.sub(r"[^\w\s]", " ", text)
        # Remove English characters and numbers.
        text = re.sub(r"[a-zA-z0-9]+", " ", text)

        text = re.sub(r" +", " ", text)
        text = text.split(" ")
        # some stupid empty tokens should be removed.
        text = [t.strip() for t in text if t.strip()]
        cleaned.append(" ".join(text))
    return cleaned


In [ ]:

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', text)




In [ ]:
#preprocess train dataset
project['تبصرے'] = project['تبصرے'].apply(remove_emoji)



In [ ]:
# Remove stop words from text
from typing import FrozenSet

# Urdu Language Stop words list
STOP_WORDS: FrozenSet[str] = frozenset("""
 آ آئی آئیں آئے آتا آتی آتے آس آمدید آنا آنسہ آنی آنے آپ آگے آہ آہا آیا اب ابھی ابے
 ارے اس اسکا اسکی اسکے اسی اسے اف افوہ البتہ الف ان اندر انکا انکی انکے انہوں انہی انہیں اوئے اور اوپر
 اوہو اپ اپنا اپنوں اپنی اپنے اپنےآپ اکثر اگر اگرچہ اہاہا ایسا ایسی ایسے ایک بائیں بار بارے بالکل باوجود باہر
 بج بجے بخیر بشرطیکہ بعد بعض بغیر بلکہ بن بنا بناؤ بند بڑی بھر بھریں بھی بہت بہتر تاکہ تاہم تب تجھ
 تجھی تجھے ترا تری تلک تم تمام تمہارا تمہاروں تمہاری تمہارے تمہیں تو تک تھا تھی تھیں تھے تیرا تیری تیرے
 جا جاؤ جائیں جائے جاتا جاتی جاتے جانی جانے جب جبکہ جدھر جس جسے جن جناب جنہوں جنہیں جو جہاں جی جیسا
 جیسوں جیسی جیسے حالانکہ حالاں حصہ حضرت خاطر خالی خواہ خوب خود دائیں درمیان دریں دو دوران دوسرا دوسروں دوسری دوں
 دکھائیں دی دیئے دیا دیتا دیتی دیتے دیر دینا دینی دینے دیکھو دیں دیے دے ذریعے رکھا رکھتا رکھتی رکھتے رکھنا رکھنی
 رکھنے رکھو رکھی رکھے رہ رہا رہتا رہتی رہتے رہنا رہنی رہنے رہو رہی رہیں رہے ساتھ سامنے ساڑھے سب سبھی
 سراسر سمیت سوا سوائے سکا سکتا سکتے سہ سہی سی سے شاید شکریہ صاحب صاحبہ صرف ضرور طرح طرف طور علاوہ عین
 فقط فلاں فی قبل قطا لئے لائی لائے لاتا لاتی لاتے لانا لانی لانے لایا لو لوجی لوگوں لگ لگا لگتا
 لگتی لگی لگیں لگے لہذا لی لیا لیتا لیتی لیتے لیکن لیں لیے لے ماسوا مت مجھ مجھی مجھے محترم محترمہ محض
 مرا مرحبا مری مرے مزید مس مسز مسٹر مطابق مل مکرمی مگر مگھر مہربانی میرا میروں میری میرے میں نا نزدیک
 نما نہ نہیں نیز نیچے نے و وار واسطے واقعی والا والوں والی والے واہ وجہ ورنہ وغیرہ ولے وگرنہ وہ وہاں
 وہی وہیں ویسا ویسے ویں پاس پایا پر پس پلیز پون پونی پونے پھر پہ پہلا پہلی پہلے پیر پیچھے چاہئے
 چاہتے چاہیئے چاہے چلا چلو چلیں چلے چناچہ چند چونکہ چکی چکیں چکے ڈالنا ڈالنی ڈالنے ڈالے کئے کا کاش کب کبھی
 کدھر کر کرتا کرتی کرتے کرم کرنا کرنے کرو کریں کرے کس کسی کسے کم کن کنہیں کو کوئی کون کونسا
 کونسے کچھ کہ کہا کہاں کہہ کہی کہیں کہے کی کیا کیسا کیسے کیونکر کیونکہ کیوں کیے کے گئی گئے گا گنا
 گو گویا گی گیا ہائیں ہائے ہاں ہر ہرچند ہرگز ہم ہمارا ہماری ہمارے ہمی ہمیں ہو ہوئی ہوئیں ہوئے ہوا
 ہوبہو ہوتا ہوتی ہوتیں ہوتے ہونا ہونگے ہونی ہونے ہوں ہی ہیلو ہیں ہے یا یات یعنی یک یہ یہاں یہی یہیں
""".split())


def remove_stopwords(text: str):
    return " ".join(word for word in text.split() if word not in STOP_WORDS)


In [ ]:
project['تبصرے'] =  project['تبصرے'].apply(remove_stopwords)

In [ ]:
# split the dataset into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(project['تبصرے'], project['جذبات'], test_size=0.1, random_state=1)#
# create a count vectorizer object for tokenization and preprocessing
cv = CountVectorizer(STOP_WORDS)#create an object to fit
cv.fit(project['تبصرے'])
# transform the training and testing data using count vectorizer object(test counts,tfidf and ngrams)
x_train_plus_count =  cv.transform(x_train)#with simple count vectors
x_test_plus_count =  cv.transform(x_test)
#for further using model +countvectors + tf-idf
tfidf_tr = TfidfTransformer()
x_train_plus_count_plus_tfidf = tfidf_tr.fit_transform(x_train_plus_count)
x_test_plus_count_plus_tfidf = tfidf_tr.transform(x_test_plus_count)

# ngram level tf-idf (model plus ngram)
tv_ngram = TfidfVectorizer(ngram_range=(1,2))
tv_ngram.fit(project['تبصرے'])
x_train_tfidf_ngram =  tv_ngram.transform(x_train)
x_test_tfidf_ngram =  tv_ngram.transform(x_test)

In [ ]:
#A class for our videomodel
def videomodel(classifier, Xtrain, Ytrain, Xtest):
    classifier.fit(Xtrain, Ytrain)
    predictions = classifier.predict(Xtest)
    print("\nAccuracy : ", accuracy_score(predictions, y_test))

    target_names = ['class 0', 'class 1']
    print(classification_report(y_test, predictions, target_names=target_names))

In [ ]:
print("\nCount Vectors used with simple Naive Bayes ")
videomodel(MultinomialNB(), x_train_plus_count, y_train, x_test_plus_count)

print("\nCount vectors and Term frequency-inverse document frequency(tf-idf) with multinomial naive bayes ")
videomodel(MultinomialNB(), x_train_plus_count_plus_tfidf, y_train, x_test_plus_count_plus_tfidf)

print("\nN-Gram Vectors alongwith Naive Bayes ")
videomodel(MultinomialNB(), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)

In [ ]:
print("\nRandom Forrest Classifier with Count Vectors: ")
videomodel(RandomForestClassifier(random_state=40,n_estimators=2000), x_train_plus_count, y_train, x_test_plus_count)

print("\nRandom Forrest Classifier with Count Vectors and using TF-IDF: ")
videomodel(RandomForestClassifier(random_state=40,n_estimators=2000), x_train_plus_count_plus_tfidf, y_train, x_test_plus_count_plus_tfidf)

print("\nRandom Forrest Classifier and just N-Gram Vectors: ")
videomodel(RandomForestClassifier(random_state=40,n_estimators=2000), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)

In [ ]:
print("\nLinearSVM and Count Vectors : ")
videomodel(SVC(kernel='linear'), x_train_plus_count, y_train, x_test_plus_count)

print("\nLinearSVM CountVectors and Term Frequency Inverse Document Frequency ")
videomodel(SVC(kernel='linear'), x_train_plus_count_plus_tfidf, y_train, x_test_plus_count_plus_tfidf)

print("\nLinearSVM with the N-Gram Vector ")
videomodel(SVC(kernel='linear'), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)


LinearSVM and Count Vectors : 

Accuracy :  0.8313140726933831
              precision    recall  f1-score   support

     class 0       0.82      0.82      0.82       514
     class 1       0.84      0.84      0.84       559

    accuracy                           0.83      1073
   macro avg       0.83      0.83      0.83      1073
weighted avg       0.83      0.83      0.83      1073


LinearSVM CountVectors and Term Frequency Inverse Document Frequency 

Accuracy :  0.8397017707362535
              precision    recall  f1-score   support

     class 0       0.86      0.79      0.83       514
     class 1       0.82      0.88      0.85       559

    accuracy                           0.84      1073
   macro avg       0.84      0.84      0.84      1073
weighted avg       0.84      0.84      0.84      1073


LinearSVM with the N-Gram Vector 

Accuracy :  0.8536812674743709
              precision    recall  f1-score   support

     class 0       0.90      0.78      0.84       514
   

Naive Bayes with count vectors curve


In [ ]:
naivebayes = MultinomialNB()
naivebayes.fit(x_train_plus_count, y_train)
predictions = naivebayes.predict(x_test_plus_count)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = naivebayes.predict_proba(x_test_plus_count)
preds = probs[:,1]
fpr_nb, tpr_nb, threshold_nb = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_nb, tpr_nb)

import matplotlib.pyplot as plt
plt.title('MultinomialNaive bayes Roc Curve 1')
plt.plot(fpr_nb, tpr_nb, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

SVM with count vectors curve


In [ ]:
svm = SVC(kernel = 'linear', probability=True)
svm.fit(x_train_plus_count, y_train)
predictions = svm.predict(x_test_plus_count)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = svm.predict_proba(x_test_plus_count)
preds = probs[:,1]
fpr_svm, tpr_svm, threshold_svm = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_svm, tpr_svm)

import matplotlib.pyplot as plt
plt.title('Linear SVM Roc Curve 1')
plt.plot(fpr_svm, tpr_svm, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

RFC with count vectors

In [ ]:
randf = RandomForestClassifier(random_state=40,n_estimators=2000)
randf.fit(x_train_plus_count, y_train)
predictions = randf.predict(x_test_plus_count)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = randf.predict_proba(x_test_plus_count)
preds = probs[:,1]
fpr_rf, tpr_rf, threshold_rf = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_rf, tpr_rf)

import matplotlib.pyplot as plt
plt.title('Random Forrest Classifier Roc Curve 1')
plt.plot(fpr_rf, tpr_rf, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

Naive Bayes with TFIDF

In [ ]:
naivebayes = MultinomialNB()
naivebayes.fit(x_train_plus_count_plus_tfidf, y_train)
predictions = naivebayes.predict(x_test_plus_count_plus_tfidf)
print("\nThe Accuracy  ", accuracy_score(predictions, y_test))

probs = naivebayes.predict_proba(x_test_plus_count_plus_tfidf)
preds = probs[:,1]
fpr_nb, tpr_nb, threshold_nb = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_nb, tpr_nb)

import matplotlib.pyplot as plt
plt.title('Multinomial naive Bayes Roc Curve 2')
plt.plot(fpr_nb, tpr_nb, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

SVM with TFIDF



In [ ]:
svm = SVC(kernel = 'linear', probability=True)
svm.fit(x_train_plus_count_plus_tfidf, y_train)
predictions = svm.predict(x_test_plus_count_plus_tfidf)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = svm.predict_proba(x_test_plus_count_plus_tfidf)
preds = probs[:,1]
fpr_svm, tpr_svm, threshold_svm = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_svm, tpr_svm)

import matplotlib.pyplot as plt
plt.title('Linear SVM Roc Curve 2')
plt.plot(fpr_svm, tpr_svm, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

RFC with TFIDF

In [ ]:

randf = RandomForestClassifier(random_state=40,n_estimators=2000)
randf.fit(x_train_plus_count_plus_tfidf, y_train)
predictions = randf.predict(x_test_plus_count_plus_tfidf)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = randf.predict_proba(x_test_plus_count_plus_tfidf)
preds = probs[:,1]
fpr_rf, tpr_rf, threshold_rf = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_rf, tpr_rf)

import matplotlib.pyplot as plt
plt.title('Random Forrest Classifier Roc Curve 2')
plt.plot(fpr_rf, tpr_rf, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

Naive Bayes with N grams curve

In [ ]:
naivebayes = MultinomialNB()
naivebayes.fit(x_train_tfidf_ngram, y_train)
predictions = naivebayes.predict(x_test_tfidf_ngram)
print("\nThe Accuracy  ", accuracy_score(predictions, y_test))

probs = naivebayes.predict_proba(x_test_tfidf_ngram)
preds = probs[:,1]
fpr_nb, tpr_nb, threshold_nb = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_nb, tpr_nb)

import matplotlib.pyplot as plt
plt.title('Multinomial naive Bayes Roc Curve 3')
plt.plot(fpr_nb, tpr_nb, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

SVM with n grams curve

RFC with N gram curve

In [ ]:
randf = RandomForestClassifier(random_state=40,n_estimators=2000)
randf.fit(x_train_tfidf_ngram, y_train)
predictions = randf.predict(x_test_tfidf_ngram)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = randf.predict_proba(x_test_tfidf_ngram)
preds = probs[:,1]
fpr_rf, tpr_rf, threshold_rf = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_rf, tpr_rf)

import matplotlib.pyplot as plt
plt.title('Random Forrest Classifier Roc Curve 3')
plt.plot(fpr_rf, tpr_rf, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()